In [55]:
import numpy as np
import sys
import glob 
import os 
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import scipy.io as sio

In [22]:
mouseName = "GF193"
nframes = 211

# Hardcoded paths to the server
datapath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\' + mouseName + "\\" 
serverpath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\'
behavpath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\BehaviourFiles\\'

# Detect the operating system to fix the slashes

if (os.name == "posix"):
    print("Linux or MacOS system detected")
    datapath = datapath.replace('\\','/')
    behavpaht = behavpath.replace('\\','/')
    serverpath = serverpath.replace('\\','/')
else:
    print("Windows system detected")

# Find the imaging days
imaging_days_path = glob.glob(os.path.join(datapath, "GF*"))

for n,f in enumerate(imaging_days_path):
    imaging_days[n] = os.path.basename(f)
    
for i,f in enumerate(imaging_days):
    
    print('Loading day : ' + imaging_days[i])
    
    F_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "F.npy"
    Fneu_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "Fneu.npy"
    iscell_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "iscell.npy"
    spks_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "spks.npy"
    
    F = np.load(F_path)
    Fneu = np.load(Fneu_path)
    spks = np.load(spks_path)
    iscell = np.load(iscell_path)
    
    F = F[iscell[:,0]==1.]
    Fneu = Fneu[iscell[:,0]==1.]
    spks = spks[iscell[:,0]==1.]
    
    print('Number of Selected ROIs : ' + str(F.shape[0])
          
          
    behav_path = behavpath + imaging_days[i] + os.sep + 'BehavResults.mat'

    print('Loading behaviour: ' + behav_path)

    beh = sio.loadmat(behav_path)

    vals = beh['BehavResults'][0,0]

    keys = beh['BehavResults'][0,0].dtype.descr

    beh = np.array(vals[keys[0][0]][:][:])
    

    
    frames_path = serverpath + imaging_days[i] + os.sep + 'TrialFrames.mat'
          
    print('Loading frame info: ' +  frames_path)

    frames = sio.loadmat(frames_path)
          
    numOfFrames = frames['NumOfFrames']

    numOfFrames = np.array(numOfFrames)
          
    
    for n in range(len(F)):
          
          
          

    
    

Windows system detected


In [130]:
CellsDict = dict.fromkeys(['day','neuron','trialType','F','Fneu','spks'])

neuron = np.empty(len(F))
trialType = [];

for t in range(len(performance)):
    
        if performance[t] == 0.0:
            
            trialType.append('WM')
            
        elif performance[t] == 1.0:
            
            trialType.append('AM')
            
        elif performance[t] == 2.0:
            
            trialType.append('WH')
         
        elif performance[t] == 3.0:
            
            trialType.append('AH')
           
        elif performance[t] == 4.0:
            
            trialType.append('CR')
            
        elif performance[t] == 5.0:
            
            trialType.append('FA')
            
        for n in range(len(F)):
            
            neuron[n] = n
       



In [131]:
trialType

['FA',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'CR',
 'AH',
 'AH',
 'AH',
 'CR',
 'FA',
 'AH',
 'FA',
 'AH',
 'CR',
 'CR',
 'CR',
 'AH',
 'AH',
 'AH',
 'CR',
 'AH',
 'FA',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'CR',
 'AH',
 'CR',
 'AH',
 'AH',
 'FA',
 'CR',
 'FA',
 'AH',
 'FA',
 'AH',
 'FA',
 'CR',
 'AH',
 'AH',
 'FA',
 'CR',
 'AH',
 'AH',
 'AH',
 'AH',
 'FA',
 'FA',
 'AH',
 'CR',
 'AH',
 'CR',
 'CR',
 'CR',
 'AH',
 'AH',
 'CR',
 'CR',
 'AM',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'CR',
 'CR',
 'AH',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'FA',
 'CR',
 'AH',
 'AM',
 'AH',
 'AH',
 'CR',
 'CR',
 'CR',
 'AH',
 'AH',
 'FA',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'CR',
 'AH',
 'CR',
 'CR',
 'CR',
 'AH',
 'AH',
 'CR',
 'AH',
 'AH',
 'FA',
 'AH',
 'CR',
 'FA',
 'AH',
 'AH',
 'FA',
 'CR',
 'AH',
 'AH',
 'CR',
 'AH',
 'CR',
 'AH',
 'CR',
 'AH',
 'FA',
 'AH',
 'AH',
 'CR',
 'CR',
 'FA',
 'CR',
 'AH',
 'AH',
 'AH',
 'CR',
 'CR',
 'AH',
 'AH',
 'AH',
 'FA',
 'FA',
 'AH',
 'FA',
 'AH',
 'CR',
 'AH',
 'CR',
 'AH',

In [81]:
CellsDict

{'day': [[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[None,
                                                                                                                                                                                                                             0],
                                                                                                                                                                                                                            1],
                                                                                                                                                                                                                           2],
                                                                                              

In [62]:
i = 1
serverpath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\'
F_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "F.npy"
iscell_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "iscell.npy"
Fneu_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "Fneu.npy"
spks_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "spks.npy"
F = np.load(F_path)
iscell = np.load(iscell_path)
spks = np.load(spks_path)
Fneu = np.load(Fneu_path)
F = F[iscell[:,0]==1.]
Fneu = Fneu[iscell[:,0]==1.]
spks = spks[iscell[:,0]==1.]

print('Number of Selected ROIs : ' + str(F.shape[0]))

behav_path = behavpath + imaging_days[i] + os.sep + 'BehavResults.mat'

print('Loading behaviour: ' + behav_path)

beh = sio.loadmat(behav_path)

vals = beh['BehavResults'][0,0]

keys = beh['BehavResults'][0,0].dtype.descr

beh = np.array(vals[keys[0][0]][:][:])

performance = beh[:,9]

frames_path = serverpath + imaging_days[i] + os.sep + 'TrialFrames.mat'

print('Loading frame info: ' +  frames_path)

frames = sio.loadmat(frames_path)

numOfFrames = frames['NumOfFrames']
    
numOfFrames = np.array(numOfFrames)

numOfFrames
#plt.figure(figsize=(30,8))
#plt.plot(F[1,:])

Number of Selected ROIs : 220
Loading behaviour: \\sv1files.epfl.ch\Petersen-Lab\data\GF193\Recordings\BehaviourFiles\GF193_21062019\BehavResults.mat
Loading frame info: \\sv1files.epfl.ch\Petersen-Lab\data\GF193\Recordings\Imaging\GF193_21062019\TrialFrames.mat


array([[ 77],
       [ 54],
       [211],
       [ 67],
       [211],
       [ 80],
       [211],
       [ 51],
       [211],
       [211],
       [211],
       [ 51],
       [ 62],
       [211],
       [ 52],
       [ 70],
       [104],
       [211],
       [ 55],
       [211],
       [ 57],
       [ 65],
       [211],
       [ 66],
       [ 66],
       [ 94],
       [108],
       [211],
       [211],
       [ 67],
       [211],
       [ 54],
       [ 52],
       [ 58],
       [211],
       [ 72],
       [106],
       [ 62],
       [105],
       [ 76],
       [211],
       [101],
       [211],
       [106],
       [ 59],
       [211],
       [211],
       [ 49],
       [ 61],
       [ 82],
       [211],
       [ 49],
       [211],
       [211],
       [211],
       [211],
       [211],
       [ 52],
       [211],
       [211],
       [ 54],
       [ 53],
       [ 85],
       [211],
       [ 54],
       [211],
       [211],
       [ 51],
       [108],
       [ 90],
       [211],
      

In [63]:
numOfFrames.shape

(580, 1)

In [64]:
performance.shape

(580,)

In [60]:
frames

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Jun 29 12:40:56 2019',
 '__version__': '1.0',
 '__globals__': [],
 'NumOfFrames': array([[ 77],
        [ 54],
        [211],
        [ 67],
        [211],
        [ 80],
        [211],
        [ 51],
        [211],
        [211],
        [211],
        [ 51],
        [ 62],
        [211],
        [ 52],
        [ 70],
        [104],
        [211],
        [ 55],
        [211],
        [ 57],
        [ 65],
        [211],
        [ 66],
        [ 66],
        [ 94],
        [108],
        [211],
        [211],
        [ 67],
        [211],
        [ 54],
        [ 52],
        [ 58],
        [211],
        [ 72],
        [106],
        [ 62],
        [105],
        [ 76],
        [211],
        [101],
        [211],
        [106],
        [ 59],
        [211],
        [211],
        [ 49],
        [ 61],
        [ 82],
        [211],
        [ 49],
        [211],
        [211],
        [211],
        [211],
      

In [45]:
F = F[iscell[:,0]==1.]

F.shape[0]

220

In [ ]:
for i,f in enumerate(imaging_days):
    
    F_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "F.npy"
    Fneu_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "Fneu.npy"
    iscell_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "iscell.npy"
    spks_path = datapath + imaging_days[i] + os.sep + 'suite2p' + os.sep + 'plane0' + os.sep + "spks.npy"
    

In [21]:
os.sep + os.sep

'\\\\'